<a href="https://colab.research.google.com/github/vgallegod/Senales-y-sistemas/blob/main/Parcial_2_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit -q #instalación de librerías
!npm install localtunnel
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 1.6 MB/s eta 0:00:00

added 22 packages, and audited 23 packages in 5s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues, run:
  npm audit fix

Run `npm audit` for details.
     - 2.7 MB 9.0 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux

In [ ]:
!pip install streamlit_option_menu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.4/823.4 kB 19.0 MB/s eta 0:00:00


In [ ]:
import os

folder_name = "pages"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

#Punto 2


In [ ]:
# Para graficar
%%writefile /content/pages/Punto_2.py

#%matplotlib inline
import streamlit as st  #librería para streamlit
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")

st.set_page_config(
    page_title="Punto2",
    page_icon="2️⃣",
)

st.title("PUNTO 2")
st.header("Simulacion y grafica espectro de Fourier")

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Parámetros de la señal
A = 1.0  # Amplitud
Fo = 5   # Frecuencia de la señal (Hz)
T = 1/Fo  # Periodo
Fs = 1000  # Frecuencia de muestreo definida
t = np.arange(-T/2, T/2, 1/Fs)  # Vector de tiempo generado
Nm = len(t)  # Número de muestras requeridas

# Definir la señal x(t) = |A * sin(2 * pi * Fo * t)|^2
x_t = (A * np.sin(2 * np.pi * Fo * t))**2

# Graficar la señal original
plt.plot(t, x_t, 'r', linewidth=2)
plt.grid()
plt.xlabel("t [s]", fontsize=14)
plt.ylabel("x(t)", fontsize=14)
plt.title("Señal x(t) = |A * sin(2 * pi * Fo * t)|^2")
st.pyplot()

# 1. Transformada de Fourier
X_f = np.fft.fft(x_t)  # FFT de la señal
freqs = np.fft.fftfreq(len(t), d=t[1]-t[0])  # Frecuencias asociadas
X_f = np.fft.fftshift(X_f)  # Centramos el espectro en torno a 0
freqs = np.fft.fftshift(freqs)  # Lo mismo para las frecuencias

# Magnitud y fase del espectro
magnitude = np.abs(X_f)  # Magnitud
phase = np.angle(X_f)    # Fase

# Graficar espectro de magnitud y fase
plt.figure(figsize=(12, 6))

# Gráfica de la magnitud del espectro
plt.subplot(2, 1, 1)
plt.plot(freqs, 20 * np.log10(magnitude + 1e-12))  # Evitar log(0)
plt.title('Espectro de Magnitud (Decibelios)')
plt.xlabel('Frecuencia (Hz)')
plt.ylabel('Magnitud (dB)')
plt.grid(True)

# Gráfica de la fase del espectro
plt.subplot(2, 1, 2)
plt.plot(freqs, phase)
plt.title('Espectro de Fase')
plt.xlabel('Frecuencia (Hz)')
plt.ylabel('Fase (radianes)')
plt.grid(True)

plt.tight_layout()
st.pyplot()

# 2. Reconstrucción de la señal con diferentes N
N_max = 50  # Número máximo de términos en la serie de Fourier
error_relativo = []

# Loop para diferentes valores de N
for N in range(1, N_max + 1):
    # Filtrar las componentes de la FFT: Mantener solo las primeras N componentes
    X_f_filtered = np.copy(X_f)
    X_f_filtered[N:] = 0  # Mantener solo las primeras N componentes

    # Reconstrucción de la señal usando la serie de Fourier inversa
    x_reconstructed = np.fft.ifft(np.fft.ifftshift(X_f_filtered))

    # Calcular el error relativo entre la señal original y la reconstruida
    error = np.linalg.norm(x_t - x_reconstructed) / np.linalg.norm(x_t)
    error_relativo.append(error)

    # Graficar la señal reconstruida para algunos valores de N
    if N in [1, 5, 10, 20, 50]:  # Graficamos para N = 1, 5, 10, 20, 50
        plt.figure()
        plt.plot(t, x_t, label='Señal Original')
        plt.plot(t, x_reconstructed, label=f'Señal Reconstruida (N={N})', linestyle='--')
        plt.title(f'Reconstrucción de la Señal con N={N}')
        plt.xlabel('Tiempo (s)')
        plt.ylabel('Amplitud')
        plt.legend()
        plt.grid(True)
        st.pyplot()

# 3. Graficar el error relativo
plt.figure()
plt.plot(range(1, N_max + 1), error_relativo)
plt.title('Error Relativo de Reconstrucción')
plt.xlabel('Número de Términos (N)')
plt.ylabel('Error Relativo')
plt.grid(True)
st.pyplot()

Overwriting /content/pages/Punto_2.py


#Punto 3

In [ ]:
!pip install soundfile

In [ ]:
#Instalacion del download de youtube
import os
import yt_dlp as youtube_dl
def download_ytvid_as_mp3(video_url,name):
    #video_url = input("enter url of youtube video:")
    video_info = youtube_dl.YoutubeDL().extract_info(url = video_url,download=False)
    filename = f"{name}.mp3"
    options={
        'format':'bestaudio/best',
        'keepvideo':False,
        'outtmpl':filename,
    }

    with youtube_dl.YoutubeDL(options) as ydl:
        ydl.download([video_info['webpage_url']])

    print("Download complete... {}".format(filename))

In [ ]:
link_= ["https://youtu.be/_fwyrbmaiAI?si=9L6e4-V8OGo3Jalp"] #Se selecciona el link de youtube de la canciòn deseada
name_ = ["Ilegales"]                                        #Nombre de la banda/artista
for name,link in zip(name_,link_):
    print(name)
    download_ytvid_as_mp3(link,name)                        #Se descarga el audio mp3 de la cancion

Ilegales
[youtube] Extracting URL: https://youtu.be/_fwyrbmaiAI?si=9L6e4-V8OGo3Jalp
[youtube] _fwyrbmaiAI: Downloading webpage
[youtube] _fwyrbmaiAI: Downloading ios player API JSON
[youtube] _fwyrbmaiAI: Downloading web creator player API JSON
[youtube] _fwyrbmaiAI: Downloading player 5f8f5b0f
[youtube] _fwyrbmaiAI: Downloading m3u8 information
[youtube] Extracting URL: https://www.youtube.com/watch?v=_fwyrbmaiAI
[youtube] _fwyrbmaiAI: Downloading webpage
[youtube] _fwyrbmaiAI: Downloading ios player API JSON
[youtube] _fwyrbmaiAI: Downloading web creator player API JSON
[youtube] _fwyrbmaiAI: Downloading m3u8 information
[info] _fwyrbmaiAI: Downloading 1 format(s): 251
[download] Destination: Ilegales.mp3
[download] 100% of    2.33MiB in 00:00:00 at 11.17MiB/s  
Download complete... Ilegales.mp3


In [ ]:
!ffmpeg -y -i Ilegales.mp3 output.wav

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
%%writefile /content/pages/Punto_3.py
#Codigo del Dashboard
import streamlit as st
import pandas as pd
import numpy as np
import os
import subprocess
import soundfile as sf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
import yt_dlp as youtube_dl
import joblib
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq

st.set_page_config(
    page_title="Punto3",
    page_icon="3️⃣",
)

st.title("Punto 3")
st.header("Frecuencia de la señal modelada en amplitud")

nombre_out = "output.wav"
audio = "/content/output.wav"
song_duration = st.slider('Segundos de duración:')
st.audio(audio, format='audio/wav', start_time=0, end_time=song_duration)
x, fs = sf.read(nombre_out)
# read speech signal from file
print('Frecuencia de muestreo %.2f[Hz]\naudio %s' % (fs,nombre_out))

# Cargar el archivo de audio
audio_signal, sampling_rate = sf.read(nombre_out)

# Verificar cuántos canales tiene el archivo
print(f"Forma del audio: {audio_signal.shape}")

# Si es estéreo (2 canales), lo convertimos en mono promediando los canales
if len(audio_signal.shape) > 1 and audio_signal.shape[1] == 2:
    audio_mono = np.mean(audio_signal, axis=1)
else:
    audio_mono = audio_signal  # Si ya es mono, no hacemos nada

# Guardar el archivo de audio en mono
sf.write('audio_mono.wav', audio_mono, sampling_rate)

audio_mono.shape

ti, tf = st.slider('Seleccione el rango de procesamiento',max_value=120,value=(20, 25))

xpro = audio_mono.copy() #copiar archivos para procesar
#ti = 20 #tiempo incio a procesar seg
#tf = 25 #tiempo final a procesar seg
xs = xpro[int(ti*fs):int((tf*fs))]

from IPython.display import Audio
ns = 120 #se reproducen los primeros ns segundos
Audio(xs[:int(fs*ns)].T,rate=fs)

audio_mono = audio_mono / np.max(np.abs(audio_mono))
xs= xs/np.max(np.abs(xs))

# Simulación de modulación por amplitud (AM)
def am_modulate(audio_signal, carrier_freq, sampling_rate, mod_index):
    t = np.arange(len(audio_signal)) / sampling_rate
    carrier = np.sin(2 * np.pi * carrier_freq * t)
    am_signal = (1 + mod_index * audio_signal) * carrier
    return t, carrier, am_signal

#Simulación de la modulación AM
carrier_freq = 100  #Frecuencia de la portadora
mod_index = float(st.number_input("Ingrese el indice de modulacion:",value=10.1))   # Índice de modulación
t, carrier, am_signal = am_modulate(audio_mono, carrier_freq, sampling_rate, mod_index)

def plot_signal(time, signal, title):
    plt.figure(num=1, figsize=(10, 4))
    plt.plot(time, signal)
    plt.title(title)
    plt.xlabel('Tiempo [s]')
    plt.ylabel('Amplitud')
    plt.grid()
    st.pyplot()

def plot_frequency(signal, sampling_rate, title):
    N = len(signal)
    yf = fft(signal)
    xf = fftfreq(N, 1 / sampling_rate)[:N // 2]
    plt.figure(num=2, figsize=(10, 4))
    plt.plot(xf, 2.0 / N * np.abs(yf[:N // 2]))
    plt.title(title)
    plt.grid()
    st.pyplot()

st.subheader("Graficas de la señal")

plot_signal(t, audio_mono, "Señal de Mensaje (Tiempo)")
plot_signal(t, carrier, "Señal Portadora (Tiempo)")
plot_signal(t, am_signal, "Señal Modulada AM (Tiempo)")

#Transformada de Fourier para mostrar en frecuencia
plot_frequency(audio_mono, sampling_rate, 'Señal de Mensaje (Frecuencia)')
plot_frequency(carrier, sampling_rate, 'Señal Portadora (Frecuencia)')
plot_frequency(am_signal, sampling_rate, 'Señal Modulada AM (Frecuencia)')

 #Reproducir los audios
st.header("Señales resultantes")
play, fin = st.slider('Seleccione el rango de Reproduccion',max_value=120,value=(20, 25))

#Señal En Mono
st.audio(xs, sample_rate=fs)

#Señal Portadora
xpro_1 = carrier.copy() #copiar archivos para procesar
#ti = 20 #tiempo incio a procesar seg
#tf = 25 #tiempo final a procesar seg
xs_1 = xpro_1[int(play*fs):int((fin*fs))]
st.audio(xs_1, sample_rate=fs)

#Señal Modulada
xpro_2= am_signal.copy() #copiar archivos para procesar
#ti = 20 #tiempo incio a procesar seg
#tf = 25 #tiempo final a procesar seg
xs_2 = xpro_2[int(play*fs):int((fin*fs))]
st.audio(xs_2, sample_rate=fs)


Overwriting /content/pages/Punto_3.py


#Punto 4

In [ ]:
%%writefile /content/pages/Punto_4.py

import streamlit as st
import pandas as pd
import numpy as np
import os
import subprocess
import soundfile as sf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
import joblib
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq

st.set_page_config(
    page_title="Punto4",
    page_icon="4️⃣",
)
st.title("PUNTO 4")
st.header("Calculo de Distorsion total de armonicos")
# Parámetros del sistema
sampling_rate = 10000  # Frecuencia de muestreo
T = 1.0 / sampling_rate  # Intervalo de tiempo
t = np.linspace(0, 1.0, sampling_rate)

# Generar señal del rectificador de onda completa
def rectificador_onda_completa(t, f, R, C=None):
    # Onda senoidal fundamental
    signal = np.abs(np.sin(2 * np.pi * f * t))

    if C is not None:
        # Respuesta para carga RC
        tau = R * C
        signal = signal * np.exp(-t / tau)

    return signal

# Calcular la FFT y los armónicos
def calcular_fft(signal, sampling_rate):
    N = len(signal)
    yf = fft(signal)
    xf = fftfreq(N, 1 / sampling_rate)[:N // 2]
    return xf, np.abs(yf[:N // 2])

# Calcular THD
def calcular_thd(yf):
    V1 = yf[1]  # El primer armónico
    harmonics = np.sqrt(np.sum(yf[2:]**2))
    thd = harmonics / V1
    return thd

# Parámetros de la simulación
st.subheader("Parámetros de la simulación")
f = 60  # Frecuencia de la señal de entrada (50 Hz)
R = float(st.number_input("Ingrese el valor de R deseado:", value=10.1))  # Valor de la resistencia (ohmios)
C = float(st.number_input("Ingrese el valor de C deseado:", value=1.1))  # Valor del condensador (faradios)

# Simulación para carga resistiva pura
signal_resistiva = rectificador_onda_completa(t, f, R)

# Simulación para carga RC en serie
signal_rc = rectificador_onda_completa(t, f, R, C)

# FFT de ambas señales
xf_res, yf_res = calcular_fft(signal_resistiva, sampling_rate)
xf_rc, yf_rc = calcular_fft(signal_rc, sampling_rate)

# Calcular THD
thd_resistiva = calcular_thd(yf_res)
thd_rc = calcular_thd(yf_rc)

st.write(f"THD para carga resistiva pura: {thd_resistiva:.4f}")
st.write(f"THD para carga RC: {thd_rc:.4f}")

# Graficar señales y su espectro
plt.figure(figsize=(12, 6))

# Señal en el dominio del tiempo
plt.subplot(2, 2, 1)
plt.plot(t[:500], signal_resistiva[:500])
plt.title("Señal con carga resistiva pura")
plt.xlabel("Tiempo [s]")
plt.ylabel("Amplitud")

plt.subplot(2, 2, 2)
plt.plot(t[:500], signal_rc[:500])
plt.title("Señal con carga RC")
plt.xlabel("Tiempo [s]")
plt.ylabel("Amplitud")

# FFT de las señales
plt.subplot(2, 2, 3)
plt.plot(xf_res, yf_res)
plt.title("FFT - Carga resistiva pura")
plt.xlabel("Frecuencia [Hz]")
plt.ylabel("Amplitud")

plt.subplot(2, 2, 4)
plt.plot(xf_rc, yf_rc)
plt.title("FFT - Carga RC")
plt.xlabel("Frecuencia [Hz]")
plt.ylabel("Amplitud")

plt.tight_layout()
st.pyplot()


Overwriting /content/pages/Punto_4.py


In [ ]:
%%writefile Parcial_2.py
import os
import streamlit as st
from streamlit_option_menu import option_menu
#import Punto_2, Punto_4

st.set_page_config(
    page_title="Parcial2",
    page_icon="💻",
)
st.write("# PARCIAL 2 ")

st.sidebar.success("Seleccione el ejercicio")

st.markdown(
    """
    ### Presentado por:
    - Valeria Gallego Duque
    - Juan Diego Garcia Quintero
    - Alejandro Bernal Valencia
"""
)

Overwriting Parcial_2.py


In [ ]:
!streamlit run Parcial_2.py &>/content/logs.txt & curl ipv4.icanhazip.com

34.86.224.157


In [ ]:
!npx localtunnel --port 8501

your url is: https://free-donkeys-draw.loca.lt
^C
